In [ ]:
import os
import pandas as pd

In [ ]:
annotations_path = r"C:\Users\fatem\OneDrive\Documents\Python-Playground\Complete-Python-Django-Data-Science-and-ML-Guide\90-jupyter-notebooks\Pandas\testing1\labels"
type_coverage = {}

df = pd.read_csv('random_sales.csv', parse_dates=['Date'], date_format='%Y-%m-%d')

def get_complexity(points):
    return 2 if len(points) > 10 else 1 if len(points) > 6 else 0

In [ ]:
max_count = 300
for filename in os.listdir(annotations_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(annotations_path, filename)
        filename_without_ext = os.path.splitext(filename)[0]

        with open(file_path, "r") as file:
            types_in_file = pd.DataFrame(columns=["type_id", "complexity", "count"])
            ok_to_add = True

            for line in file:
                type_id, *points = line.split()

                complexity = get_complexity(points)

                if (
                    (types_in_file["type_id"] == type_id)
                    & (types_in_file["complexity"] == complexity)
                ).any():
                    current_count = types_in_file.loc[
                        (types_in_file["type_id"] == type_id)
                        & (types_in_file["complexity"] == complexity),
                        "count",
                    ].values[0]
                    if current_count < max_count:
                        types_in_file.loc[
                            (types_in_file["type_id"] == type_id)
                            & (types_in_file["complexity"] == complexity),
                            "count",
                        ] += 1
                    else:
                        ok_to_add = False
                        break
                else:
                    types_in_file = pd.concat(
                        [
                            types_in_file,
                            pd.DataFrame(
                                [
                                    {
                                        "type_id": type_id,
                                        "complexity": complexity,
                                        "count": 1,
                                    }
                                ]
                            ),
                        ],
                        ignore_index=True,
                    )
            if ok_to_add:
                type_coverage[filename_without_ext] = types_in_file


In [ ]:

type_mapping = {
    0: "Channel Condition - 1",
    1: "Channel Condition - 3",
    2: "Concrete Kerb",
    3: "Drain",
    4: "Driveway Condition - 1",
    5: "Intersection Crossing",
    6: "Kerb Condition - 1",
    7: "Kerb Condition - 2",
    8: "Kerb Condition - 3",
    9: "Kerb Condition - 4",
    10: "Kerb Condition - 5",
    11: "Kerb Inlet",
    12: "Ramp",
}



In [ ]:

analysis_path = r"C:\Users\fatem\OneDrive\Documents\Python-Playground\Complete-Python-Django-Data-Science-and-ML-Guide\90-jupyter-notebooks\Pandas\testing1\analysis"
file_path = os.path.join(analysis_path, "type_coverage.xlsx")
rows = []
for filename, defect_type_data in type_coverage.items():
    for _, row in defect_type_data.iterrows():
        type_label = type_mapping[int(row["type_id"])]
        rows.append(
            {
                "filename": filename,
                "type": type_label,
                "complexity": row["complexity"],
                "count": row["count"],
            }
        )
if rows:
    df = pd.DataFrame(rows)
    df.to_excel(file_path, index=False)
else:
    raise ValueError("No data to export")
# type_id, *points = line.split()

# if type_id in types_in_file:
#     types_in_file[type_id] += 1
# else:
#     types_in_file[type_id] = 1

# type_coverage[filename_without_ext] = types_in_file
